# TimeAttentionレイヤを実装する

In [1]:
import numpy as np
from common.layers import Softmax

### [演習]
* 以下のWeightSum, AttentionWeight, Attentionの, TimeAttentionクラスを完成させましょう

In [2]:
class WeightSum:
    def __init__(self):
        self.params, self.grads = [], []
        self.cache = None

    def forward(self, hs, a):
        """
        順伝播
        hs : エンコーダの中間状態
        a : アテンション荷重
        """
        N, T, H = hs.shape

        # アテンション荷重の行列を3次元配列に変形する
        ar = a.reshape(N, T, 1)#.repeat(T, axis=1)   ブロードキャストを明示的に行いたい場合はrepeatを付ける
        # エンコーダの中間状態にアテンション荷重をかけて、それを足し合わせることによって、加重平均を求める
        t = hs * ar
        c = np.sum(t, axis=1)

        self.cache = (hs, ar)
        return c  # エンコーダの中間状態を加重平均した結果

    def backward(self, dc):
        """
        逆伝播
        """
        hs, ar = self.cache
        N, T, H = hs.shape
        dt = dc.reshape(N, 1, H).repeat(T, axis=1)
        dar = dt * hs
        dhs = dt * ar
        da = np.sum(dar, axis=2)

        return dhs, da


class AttentionWeight:
    """
    アテンション荷重を算出するクラス
    """
    def __init__(self):
        self.params, self.grads = [], []
        self.softmax = Softmax()
        self.cache = None

    def forward(self, hs, h):
        """
        順伝播
        アテンション荷重を求める
        hs : エンコーダの全ての中間状態
        h : デコーダのある場所の中間状態
        """
        N, T, H = hs.shape

        #　デコーダのある場所の中間状態を3次元配列に変形する
        hr = h.reshape(N, 1, H)#.repeat(T, axis=1)
        
        # エンコーダの中間状態とコーダの中間状態を掛けて足し合わせることで内積をとる
        # 他の実装例として、hsとhrを結合し、重みWを掛けるという方法もある
        t = hs * hr
        s = np.sum(t, axis=2)
        
        # ソフトマックス関数に通すことで、正規化する
        a = self.softmax.forward(s) # アテンション重みベクトルを並べた行列 (N * T)

        self.cache = (hs, hr)
        return a

    def backward(self, da):
        """
        逆伝播
        """
        hs, hr = self.cache
        N, T, H = hs.shape

        ds = self.softmax.backward(da)
        dt = ds.reshape(N, T, 1).repeat(H, axis=2)
        dhs = dt * hr
        dhr = dt * hs
        dh = np.sum(dhr, axis=1)

        return dhs, dh


class Attention:
    """
    アテンション
    """
    def __init__(self):
        self.params, self.grads = [], []
        
        # レイヤの定義
        self.attention_weight_layer = AttentionWeight()
        self.weight_sum_layer = WeightSum()
        self.attention_weight = None

    def forward(self, hs, h):
        """
        順伝播
        hs : エンコーダの中間状態
        h : デコーダの中間状態
        """
        # アテンション荷重を求める
        a = self.attention_weight_layer.forward(hs, h)
        
        # エンコーダの中間状態にアテンション荷重をかける
        out = self.weight_sum_layer.forward(hs, a)
        self.attention_weight = a
        
        return out # エンコーダの中間状態を加重平均した結果

    def backward(self, dout):
        """
        逆伝播
        """
        dhs0, da = self.weight_sum_layer.backward(dout)
        dhs1, dh = self.attention_weight_layer.backward(da)
        dhs = dhs0 + dhs1
        return dhs, dh


class TimeAttention:
    """
    アテンションレイヤを時間方向にまとめるレイヤ
    """
    def __init__(self):
        self.params, self.grads = [], []
        self.layers = None
        self.attention_weights = None

    def forward(self, hs_enc, hs_dec):
        """
        順伝播
        hs_enc : エンコーダの中間状態
        hs_dec : デンコーダの中間状態
        """
        N, T, H = hs_dec.shape
        out = np.empty_like(hs_dec)
        self.layers = []
        self.attention_weights = []

        for t in range(T):
            """
            出力単語数分を繰り返す
            """
            layer = Attention()
            out[:, t, :] = layer.forward(hs_enc, hs_dec[:,t,:]) 
            self.layers.append(layer)
            self.attention_weights.append(layer.attention_weight)

        return out

    def backward(self, dout):
        """
        逆伝播
        dout : 勾配
        """
        N, T, H = dout.shape
        dhs_enc = 0
        dhs_dec = np.empty_like(dout)

        for t in range(T):
            """
            出力単語数分を繰り返す
            """
            layer = self.layers[t]
            dhs, dh = layer.backward(dout[:, t, :])
            dhs_enc += dhs
            dhs_dec[:,t,:] = dh

        return dhs_enc, dhs_dec


In [3]:
# 中間層ノード数
H = 4
# データ数
N = 3
# 単語数
T = 5


# モデル構築
ta = TimeAttention()

hs_enc = np.random.randn(N*T*H).reshape(N, T, H)
hs_dec =  np.random.randn(N*T*H).reshape(N, T, H)
print("hs_enc=", hs_enc)
print()
print("hs_dec=", hs_dec)
print()

# 順伝播計算
out = ta.forward(hs_enc, hs_dec)
print("out=", out)
print()

# 逆伝播計算
dout = np.random.randn(N*T*H).reshape(N, T, H)
dhs_enc, dhs_dec = ta.backward(dout)
print("dhs_enc=", dhs_enc)
print()
print("dhs_dec=", dhs_dec)
print()

hs_enc= [[[ 1.41430217e+00  1.13878311e-01  2.44006597e-01  2.59371489e-01]
  [ 1.54032089e+00  7.60391359e-01  2.44790136e-01 -8.17827828e-01]
  [ 2.18809238e+00 -4.82706152e-01 -2.49741945e-01  1.10393351e+00]
  [-2.22340038e+00 -2.51336007e+00  1.11451883e+00  2.33660432e-01]
  [ 3.11245209e-01 -5.46558462e-02  1.59272527e+00  1.07525619e+00]]

 [[-2.94727252e-01  9.32177685e-01  1.17516604e+00  3.02637329e-01]
  [-2.00308985e+00  8.96635601e-01  1.01508164e+00  5.25394680e-01]
  [ 2.12487554e-01  1.19447483e+00  5.38546378e-04  5.30954619e-01]
  [ 9.48132486e-01  8.07647900e-01 -5.77730976e-01 -1.17354219e+00]
  [ 1.75851723e+00 -1.54123466e-01 -1.38176691e+00 -1.01301031e+00]]

 [[ 1.46454933e+00 -1.32597953e+00 -9.48238570e-01 -9.46499009e-01]
  [-6.38576316e-01 -7.68174388e-01 -6.46145940e-02 -1.56408939e-01]
  [-1.85958116e-01 -1.07923193e+00  1.70081196e+00  1.36167020e+00]
  [-2.84902409e-01 -1.25893225e+00 -3.00444374e-01 -1.09670746e+00]
  [-1.99956860e-01 -4.15487161e-01 -